# Create Policy Tags from DLP Scan Results

In [ ]:
%env GOOGLE_APPLICATION_CREDENTIALS=

In [ ]:
%env CLOUDSDK_PYTHON=

In [ ]:
from google.cloud import bigquery

## Get DLP results from BigQuery

See tutorial from https://cloud.google.com/dlp/docs/querying-findings and https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries

Here is the schema of the table.

![](schema.png)

Here is the InfoTypes that were scanned.

![](dlp_scan.png)

In [ ]:
client = bigquery.Client()

In [ ]:
query_job = client.query(
    """
    SELECT
      table_counts.field_name,
      STRING_AGG( CONCAT(" ",table_counts.name," [count: ",CAST(table_counts.count_total AS String),"]")
      ORDER BY
        table_counts.count_total DESC) AS infoTypes
    FROM (
      SELECT
        locations.record_location.field_id.name AS field_name,
        info_type.name,
        COUNT(*) AS count_total
      FROM
        `sarun-project.dsi_dataset.dlp_googleapis_2021_02_07_8865834336597775185`,
        UNNEST(location.content_locations) AS locations
      WHERE
        (likelihood = 'LIKELY'
          OR likelihood = 'VERY_LIKELY'
          OR likelihood = 'POSSIBLE')
      GROUP BY
        locations.record_location.field_id.name,
        info_type.name
      HAVING
        count_total>20 ) AS table_counts
    GROUP BY
      table_counts.field_name
    ORDER BY
      table_counts.field_name
    """
)

results = query_job.result()  # Waits for job to complete.

In [ ]:
col = []
val = []
for row in results:
    col.append(row[0])
    val.append(row[1])
    print(row)

Get columns to tag

In [ ]:
tags = {k:v.split(',')[0].strip().split(' ')[0] for k, v in zip(col, val)}
tags

## Assign Policy Tags to BigQuery Columns

First, create Taxonomy using console. Then, run the following lines.

In [ ]:
from google.cloud import datacatalog_v1beta1

In [ ]:
client = datacatalog_v1beta1.PolicyTagManagerClient()

In [ ]:
name = client.taxonomy_path('sarun-project', 'us', '3573570058507175677')

In [ ]:
policy_tags = {element.display_name:element.name 
               for element in client.list_policy_tags(parent=name)}

In [ ]:
table = 'sarun-project:dsi_dataset.org_expedite_intuitive_paradigms_3de0'

In [ ]:
!bq show --schema --format=prettyjson $table > schema.json

In [ ]:
import json

In [ ]:
with open("schema.json", "r") as f:
    schema_dict = json.loads(f.read())

print(schema_dict)

In [ ]:
for field in schema_dict:
    if field['name'] in policy_tags.keys():
        policy_tag = policy_tags[field['name']]
        field['policyTags'] = {'names':[ policy_tag ]}

In [ ]:
schema_dict

In [ ]:
with open('new_schema.json', 'w') as outfile:
    json.dump(schema_dict, outfile)

In [ ]:
!bq update $table new_schema.json

Here is the new schema of the table.

![](new_schema.png)